In [1]:
import numpy as np
import matplotlib.pyplot as plt
from spiker import *
import pickle
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Tensorflow model
Winl1, Wl1l2, Wl2out, Bl1, Bl2, Bout = pickle.load( open( "weights.dat", "rb" ) )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [2]:
def compute_layer(ip, N, W, B = 0):
    #print(W.shape, ip.shape) 
    x = np.dot(W,ip) + np.dot(B,scale_factor)
    return N.compute(x)

In [3]:
# Simulation setting parameters
Ts = 1e-6
tstart = 0
tstop = 0.02
scale_factor = 1e-9
tseries = gen_time(tstart,tstop,Ts)
time = tseries.time

In [4]:
# Spiking simulations
nip,nnip,nnl1,nnl2,nnop = (784,784,1000,256,10)

Neuronip = Neuron(nnip,Ts=Ts)
Neuronl1 = Neuron(nnl1,Ts=Ts)
Neuronl2 = Neuron(nnl2,Ts=Ts)
Neuronop = Neuron(nnop,Ts=Ts)
Y  = Vsyn(nnop,Ts=Ts)    # Readout to LPF neuron output

Win = np.eye(nip)
Winl1, Wl1l2, Wl2out, Bl1, Bl2, Bout = pickle.load( open( "weights.dat", "rb" ) )
Winl1, Wl1l2, Wl2out, Bl1, Bl2, Bout = Winl1.T, Wl1l2.T, Wl2out.T, Bl1.T, Bl2.T, Bout.T
Wop = np.eye(nnop)
        
count = nnop
def run_test(test_vector, test_ans):
    output = np.zeros((count,len(tseries.time)))
    for index in range(len(tseries.time)):
        ip = test_vector[:,index] * scale_factor
        
        ip_layer  = compute_layer(ip,Neuronip, Win)
        l1_layer  = compute_layer(ip_layer,Neuronl1,Winl1,Bl1)
        l2_layer  = compute_layer(l1_layer,Neuronl2,Wl1l2,Bl2)
        out_layer = compute_layer(l2_layer,Neuronop,Wl2out,Bout)
        Yop       = Y.compute(out_layer)
      
    output[:,index] = Y.states
    correct_prediction = np.sum(np.argmax(output[:,-1]) == np.argmax(test_ans)) == 1
    print("Correct",np.argmax(output[:,-1]),"Result",np.argmax(test_ans))
    return correct_prediction

In [ ]:
numtests = 100
correct = 0
for sample in range(numtests):
    test_ip,test_op = mnist.train.next_batch(1)
    test_ip = test_ip > 0
    test_ip.shape = (784,)
    test_vector = np.repeat(test_ip[:, np.newaxis], len(time), axis=1)
    correct = correct + run_test(test_vector,test_op)
    print("Pass count", correct,'/',sample+1, end='\r')

Correct 0 Result 6
Correct 0 Result 3
Correct 0 Result 9
Correct 0 Result 9
Correct 0 Result 2
Correct 0 Result 1
Correct 0 Result 7
Correct 0 Result 2
Correct 0 Result 9
Correct 0 Result 8
Correct 0 Result 6
Correct 0 Result 7
Correct 0 Result 8
Correct 0 Result 7
Correct 2 Result 1
Correct 9 Result 8
Correct 5 Result 5
Correct 9 Result 9
Correct 9 Result 5
Correct 2 Result 9
Correct 5 Result 3
Correct 1 Result 2
Correct 6 Result 2
Correct 6 Result 9
Correct 2 Result 5
Correct 4 Result 6
Correct 0 Result 1
